In [1]:
import os
import sqlite3
from pathlib import Path
import pandas as pd

# 1) Проверь текущую директорию
print("CWD:", os.getcwd())

# 2) Попробуем найти файл рекурсивно от корня проекта
project_root = Path(r"E:\PythonProjects\ItmoProjects")  # поправь при необходимости
candidates = list(project_root.rglob("cs2_market.db"))
if not candidates:
    raise FileNotFoundError("Не нашёл cs2_market.db. Проверь путь/папку 'ТХБД'.")

db_path = candidates[0]
print("DB path:", db_path)

# 3) Открываем в режиме read-only (надёжнее на Windows)
con = sqlite3.connect(f"file:{db_path.as_posix()}?mode=ro", uri=True)

# 4) Что за таблицы
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", con
)
print("Таблицы:")
print(tables)

# 5) Пару строк из таблиц
if (tables["name"] == "skinport_items").any():
    df_items = pd.read_sql_query(
        "SELECT market_hash_name, currency, suggested_price, quantity "
        "FROM skinport_items LIMIT 5;", con
    )
    print("\nskinport_items (5 строк):")
    print(df_items)

if (tables["name"] == "skinport_sales_agg").any():
    df_hist = pd.read_sql_query(
        "SELECT market_hash_name, period, min, max, avg, median, volume "
        "FROM skinport_sales_agg LIMIT 5;", con
    )
    print("\nskinport_sales_agg (5 строк):")
    print(df_hist)



CWD: e:\PythonProjects\ItmoProjects\TXDB
DB path: E:\PythonProjects\ItmoProjects\TXDB\cs2_market.db
Таблицы:
                    name
0  bitskins_recent_sales
1         skinport_items
2     skinport_sales_agg

skinport_items (5 строк):
                   market_hash_name currency  suggested_price  quantity
0  10 Year Birthday Sticker Capsule      EUR             0.87       176
1      1st Lieutenant Farlow | SWAT      EUR            16.07       163
2              2020 RMR Challengers      EUR             0.27       477
3               2020 RMR Contenders      EUR             0.38       465
4                  2020 RMR Legends      EUR             0.29       541

skinport_sales_agg (5 строк):
                                    market_hash_name         period   min  \
0  Souvenir Charm | Austin 2025 Highlight | Quad ...  last_24_hours  None   
1  Souvenir Charm | Austin 2025 Highlight | Quad ...    last_7_days  None   
2  Souvenir Charm | Austin 2025 Highlight | Quad ...   last_30_days  N

In [4]:
print(pd.read_sql_query("SELECT count(*) FROM skinport_items;", con))

   count(*)
0     25556


In [7]:
pd.read_sql_query("SELECT * FROM skinport_sales_agg;", con)

,market_hash_name,currency,item_page,market_page,period,min,max,avg,median,volume,extracted_at
0,Souvenir Charm | Austin 2025 Highlight | Quad ...,EUR,https://skinport.com/item/souvenir-charm-austi...,https://skinport.com/market?item=Austin%202025...,last_24_hours,NaN,NaN,NaN,NaN,0,2025-10-17 14:11:34
1,Souvenir Charm | Austin 2025 Highlight | Quad ...,EUR,https://skinport.com/item/souvenir-charm-austi...,https://skinport.com/market?item=Austin%202025...,last_7_days,NaN,NaN,NaN,NaN,0,2025-10-17 14:11:34
2,Souvenir Charm | Austin 2025 Highlight | Quad ...,EUR,https://skinport.com/item/souvenir-charm-austi...,https://skinport.com/market?item=Austin%202025...,last_30_days,NaN,NaN,NaN,NaN,0,2025-10-17 14:11:34
3,Souvenir Charm | Austin 2025 Highlight | Quad ...,EUR,https://skinport.com/item/souvenir-charm-austi...,https://skinport.com/market?item=Austin%202025...,last_90_days,NaN,NaN,NaN,NaN,0,2025-10-17 14:11:34
4,★ M9 Bayonet | Doppler (Minimal Wear),EUR,https://skinport.com/item/m9-bayonet-doppler-m...,https://skinport.com/market?item=Doppler&cat=K...,last_24_hours,NaN,NaN,NaN,NaN,0,2025-10-17 14:11:34
...,...,...,...,...,...,...,...,...,...,...,...
104419,MP5-SD | Focus (Factory New),EUR,https://skinport.com/item/mp5-sd-focus-factory...,https://skinport.com/market?item=Focus&cat=SMG...,last_90_days,1.04,3.25,2.06,2.0,5,2025-10-17 14:11:34
104420,MAC-10 | Cat Fight (Factory New),EUR,https://skinport.com/item/mac-10-cat-fight-fac...,https://skinport.com/market?item=Cat%20Fight&c...,last_24_hours,NaN,NaN,NaN,NaN,0,2025-10-17 14:11:34
104421,MAC-10 | Cat Fight (Factory New),EUR,https://skinport.com/item/mac-10-cat-fight-fac...,https://skinport.com/market?item=Cat%20Fight&c...,last_7_days,5.44,8.84,6.67,6.2,4,2025-10-17 14:11:34
104422,MAC-10 | Cat Fight (Factory New),EUR,https://skinport.com/item/mac-10-cat-fight-fac...,https://skinport.com/market?item=Cat%20Fight&c...,last_30_days,5.44,8.84,6.67,6.2,4,2025-10-17 14:11:34
